# Desafio 5

Neste desafio, vamos praticar sobre redução de dimensionalidade com PCA e seleção de variáveis com RFE. Utilizaremos o _data set_ [Fifa 2019](https://www.kaggle.com/karangadiya/fifa19), contendo originalmente 89 variáveis de mais de 18 mil jogadores do _game_ FIFA 2019.

> Obs.: Por favor, não modifique o nome das funções de resposta.

## _Setup_ geral

In [108]:
from math import sqrt

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sct
import seaborn as sns
import statsmodels.api as sm
import statsmodels.stats as st
from sklearn.decomposition import PCA

from loguru import logger

In [109]:
# Algumas configurações para o matplotlib.
#%matplotlib inline

from IPython.core.pylabtools import figsize


figsize(12, 8)

sns.set()

In [110]:
fifa = pd.read_csv("fifa.csv")

In [111]:
columns_to_drop = ["Unnamed: 0", "ID", "Name", "Photo", "Nationality", "Flag",
                   "Club", "Club Logo", "Value", "Wage", "Special", "Preferred Foot",
                   "International Reputation", "Weak Foot", "Skill Moves", "Work Rate",
                   "Body Type", "Real Face", "Position", "Jersey Number", "Joined",
                   "Loaned From", "Contract Valid Until", "Height", "Weight", "LS",
                   "ST", "RS", "LW", "LF", "CF", "RF", "RW", "LAM", "CAM", "RAM", "LM",
                   "LCM", "CM", "RCM", "RM", "LWB", "LDM", "CDM", "RDM", "RWB", "LB", "LCB",
                   "CB", "RCB", "RB", "Release Clause"
]

try:
    fifa.drop(columns_to_drop, axis=1, inplace=True)
except KeyError:
    logger.warning(f"Columns already dropped")

## Inicia sua análise a partir daqui

In [112]:
fifa.head()

,Age,Overall,Potential,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,...,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes
0,31,94,94,84.0,95.0,70.0,90.0,86.0,97.0,93.0,...,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0
1,33,94,94,84.0,94.0,89.0,81.0,87.0,88.0,81.0,...,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0
2,26,92,93,79.0,87.0,62.0,84.0,84.0,96.0,88.0,...,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0
3,27,91,93,17.0,13.0,21.0,50.0,13.0,18.0,21.0,...,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0
4,27,91,92,93.0,82.0,55.0,92.0,82.0,86.0,85.0,...,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0


In [113]:
fifa.shape

(18207, 37)

In [114]:
#criando um dataframe para analise dos dados
analise=pd.DataFrame({'colunas': fifa.columns,
                     'tipos': fifa.dtypes,
                     'missing': fifa.isna().sum(),
                      'size': fifa.shape[0],
                     'unicos': fifa.nunique()})
analise['percentual']=analise['missing']/analise['size']
analise

,colunas,tipos,missing,size,unicos,percentual
Age,Age,int64,0,18207,29,0.000000
Overall,Overall,int64,0,18207,48,0.000000
Potential,Potential,int64,0,18207,47,0.000000
Crossing,Crossing,float64,48,18207,89,0.002636
Finishing,Finishing,float64,48,18207,93,0.002636
HeadingAccuracy,HeadingAccuracy,float64,48,18207,91,0.002636
ShortPassing,ShortPassing,float64,48,18207,85,0.002636
Volleys,Volleys,float64,48,18207,87,0.002636
Dribbling,Dribbling,float64,48,18207,94,0.002636
Curve,Curve,float64,48,18207,89,0.002636


In [115]:
fifa.dropna(inplace=True)

## Questão 1

Qual fração da variância consegue ser explicada pelo primeiro componente principal de `fifa`? Responda como um único float (entre 0 e 1) arredondado para três casas decimais.

In [116]:
pca=PCA(n_components=1)

In [117]:
pca.fit(fifa)

PCA(n_components=1)

In [118]:
round(pca.explained_variance_ratio_[0],3)

0.565

In [119]:
def q1():
    return round(pca.explained_variance_ratio_[0],3)

## Questão 2

Quantos componentes principais precisamos para explicar 95% da variância total? Responda como un único escalar inteiro.

In [120]:
pca_095=PCA(0.95)

In [121]:
n_comp=pca_095.fit(fifa)

In [122]:
len(n_comp.explained_variance_ratio_)

15

In [123]:
def q2():
    return len(n_comp.explained_variance_ratio_)

## Questão 3

Qual são as coordenadas (primeiro e segundo componentes principais) do ponto `x` abaixo? O vetor abaixo já está centralizado. Cuidado para __não__ centralizar o vetor novamente (por exemplo, invocando `PCA.transform()` nele). Responda como uma tupla de float arredondados para três casas decimais.

In [124]:
x = [0.87747123,  -1.24990363,  -1.3191255, -36.7341814,
     -35.55091139, -37.29814417, -28.68671182, -30.90902583,
     -42.37100061, -32.17082438, -28.86315326, -22.71193348,
     -38.36945867, -20.61407566, -22.72696734, -25.50360703,
     2.16339005, -27.96657305, -33.46004736,  -5.08943224,
     -30.21994603,   3.68803348, -36.10997302, -30.86899058,
     -22.69827634, -37.95847789, -22.40090313, -30.54859849,
     -26.64827358, -19.28162344, -34.69783578, -34.6614351,
     48.38377664,  47.60840355,  45.76793876,  44.61110193,
     49.28911284
]

In [125]:
pca2=PCA(n_components=2)

In [126]:
pca2.fit(fifa)

PCA(n_components=2)

In [127]:
pca2.components_

array([[-6.16388751e-03, -3.70623864e-02, -2.27401748e-02,
        -2.13639023e-01, -1.98891213e-01, -1.70828950e-01,
        -1.80309140e-01, -1.87038764e-01, -2.33139606e-01,
        -2.07690956e-01, -1.84764187e-01, -1.67186902e-01,
        -2.12972623e-01, -1.38740617e-01, -1.34902279e-01,
        -1.38433521e-01, -4.91611013e-02, -1.16410947e-01,
        -1.95840156e-01, -4.07181861e-02, -1.72711671e-01,
        -2.33585866e-02, -2.16594765e-01, -1.53339724e-01,
        -1.50834334e-01, -2.24571087e-01, -1.29586783e-01,
        -1.62548283e-01, -1.01038031e-01, -1.53454113e-01,
        -1.59617493e-01, -1.47955869e-01,  2.06147192e-01,
         1.96645602e-01,  1.91129889e-01,  1.97401130e-01,
         2.08994083e-01],
       [ 8.87203494e-03,  1.58367355e-04, -7.78142440e-03,
        -4.43084573e-02, -2.57629630e-01,  1.18911964e-01,
         1.21869793e-02, -1.91182282e-01, -1.18898465e-01,
        -1.27744634e-01, -1.00178915e-01,  4.89136910e-02,
        -5.12678591e-02, -9.84

In [128]:
coord_x=pca2.components_.dot(x)
coord_x0=round(coord_x[0],3)
coord_x1=round(coord_x[1],3)
tuple([coord_x0,coord_x1])

(186.556, -6.592)

In [129]:
def q3():
    return tuple([coord_x0,coord_x1])

## Questão 4

Realiza RFE com estimador de regressão linear para selecionar cinco variáveis, eliminando uma a uma. Quais são as variáveis selecionadas? Responda como uma lista de nomes de variáveis.

In [130]:
y_train = fifa['Overall']
X_train = fifa.drop(columns = 'Overall')

In [131]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

In [132]:
reg = LinearRegression()

In [133]:
rfe = RFE(reg,n_features_to_select=5)

In [134]:
rfe.fit(X_train, y_train)

RFE(estimator=LinearRegression(), n_features_to_select=5)

In [135]:
features=pd.DataFrame({'coluna':X_train.columns,
              'bool': rfe.get_support()})

In [136]:
list(features[features['bool'] == True]['coluna'])

['Age', 'Potential', 'BallControl', 'Reactions', 'GKReflexes']

In [137]:
def q4():
    return list(features[features['bool'] == True]['coluna'])

In [138]:
q1()

0.565

In [139]:
q2()

15

In [140]:
q3()

(186.556, -6.592)

In [141]:
q4()

['Age', 'Potential', 'BallControl', 'Reactions', 'GKReflexes']